<a href="https://colab.research.google.com/github/mon-project-repos/ReviewGenerator/blob/master/ReviewGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Preliminaries

## GPU Session

In [12]:
# set up session 
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)


import os
import tensorflow as tf
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
TPU address is grpc://10.104.126.154:8470


## Imports

In [13]:
import tensorflow as tf

import pickle
import pandas as pd
import numpy as np
import re

import nltk
# nltk.download('stopwords')
nltk.download('punkt')

from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data

In [14]:
dta = pickle.load(open("gdrive/My Drive/dta_review.pkl",'rb'))

dta = dta.loc[dta['prod_total']>100]



# dta['orig_rating'] = dta['rating']
# dta['rating'] = [1 if i > 0.6 else 0 for i in dta['orig_rating']]



# prod_unique = dta.product_id.unique()
# newprodid = pd.concat( [pd.Series(prod_unique) , pd.Series(range( len(prod_unique)) )], axis=1)
# newprodid.columns = ['product_id','new_product_id']
# dta = pd.merge(dta, newprodid, how='left', on='product_id')


# class_unique = dta.text_class_id.unique()
# newclassid = pd.concat([pd.Series(class_unique), pd.Series(range(len(class_unique)))], axis=1)
# newclassid.columns = ['text_class_id', 'new_class_id']
# dta = pd.merge(dta, newclassid, how='left', on='text_class_id')


# reviewer_unique = dta.reviewer_id.unique()
# newreviewerid = pd.concat([pd.Series(reviewer_unique),
#                            pd.Series(range(len(reviewer_unique)))], axis=1)
# newreviewerid.columns = ['reviewer_id', 'new_reviewer_id']
# dta = pd.merge(dta, newreviewerid, how='left', on='reviewer_id')


# review_unique = dta.review_id.unique()
# newreviewid = pd.concat([pd.Series(review_unique),
#                            pd.Series(range(len(review_unique)))], axis=1)
# newreviewid.columns = ['review_id', 'new_review_id']
# dta = pd.merge(dta, newreviewid, how='left', on='review_id')

#dta = dta.drop(columns=['id_x','id_y'])
#print(d


print('Total',dta.shape)
# print('Products', len(dta.product_id.unique()))
# print('Reviewers', len(dta.reviewer_id.unique()))
# print('Reviews', len(dta.review_id.unique()))


corpus = dta['review_data']
dta = None

Total (81581, 13)


# NLP preprocessing

## User and Product Documents

In [15]:
len(corpus)
percent_validation = .2
last_train = int(len(corpus)*(1-percent_validation))
print(corpus[last_train-1])
corpus_train, corpus_validation = corpus[:last_train], corpus[last_train:]
corpus = None

I now have two pairs. One I wear instead of slippers around the house, often as slip-on clogs, and the others I wear outdoors. These are well-cushioned for a very comfortable all day wear. Being that the stretch fabric is very flexible, these shoes should not be used for sports as your heels can  slide sideways off the sole especially in tennis or even romping with dog or on uneven ground 


In [16]:
corpus_train = ("<EOR>.  ").join(corpus_train)
sents_train = nltk.sent_tokenize(corpus_train)
len(sents_train)



208890

In [24]:
  
#remove_filters='"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n' # don't remove anything as they may be emojis
max_vocab_words = 500
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = max_vocab_words+1,
                                                  lower=True, 
                                                  filters="",
                                                 oov_token="<OOV>",char_level=False)



tokenizer.fit_on_texts(sents_train)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(len(tokenizer.word_index))
print(total_words)

{'<OOV>': 1, 'i': 2, '<eor>.': 3, 'the': 4, 'and': 5, 'a': 6, 'to': 7, 'they': 8, 'my': 9, 'these': 10, 'for': 11, 'are': 12, 'of': 13, 'in': 14, 'shoes': 15, 'have': 16, 'but': 17, 'them': 18, 'love': 19, 'very': 20, 'is': 21, 'on': 22, 'so': 23, 'comfortable': 24, 'fit': 25, 'not': 26, 'this': 27, 'with': 28, 'great': 29, 'size': 30, 'as': 31, 'that': 32, 'it': 33, 'shoe': 34, 'was': 35, 'wear': 36, 'like': 37, 'feet': 38, 'pair': 39, 'good': 40, 'just': 41, 'were': 42, 'be': 43, 'had': 44, 'shoes.': 45, 'all': 46, 'would': 47, 'at': 48, 'will': 49, 'comfortable.': 50, 'too': 51, 'you': 52, 'bought': 53, 'ordered': 54, 'them.': 55, 'than': 56, 'am': 57, 'really': 58, 'more': 59, 'little': 60, 'or': 61, 'get': 62, 'when': 63, 'up': 64, 'if': 65, 'because': 66, 'no': 67, 'buy': 68, 'look': 69, 'support': 70, 'walking': 71, 'perfect': 72, 'me': 73, 'feel': 74, 'an': 75, 'only': 76, 'wide': 77, "i'm": 78, 'shoe.': 79, 'super': 80, 'running': 81, 'out': 82, 'nice': 83, 'other': 84, 'work'

In [25]:
sents_train[1]

'All have been great shoes with the exception of the Scram which are technically trail runners but they were just to narrow in the toe box.'

In [0]:
#("/n").join(list(dta.iloc[1:5]['review_data']))


# docByProd = dta.groupby('new_product_id').apply(lambda x: (". ").join(list(x['review_data'])) )
# docByUser = dta.groupby('new_reviewer_id').apply(lambda x:  (". ").join(list(x['review_data'])) )

# print('length of prod docs', len(docByProd), 
#       '.  per product: ', max([len(x.split(" ")) for x in docByProd]),
#       ', min= ',min([len(x.split(" ")) for x in docByProd]),
#      ', mean= ', np.mean([len(x.split(" ")) for x in docByProd]),
#       ', median ', np.median([len(x.split(" ")) for x in docByProd])
#      )

# print('length of user docs', len(docByUser),
#     '.  per user: max=',max([len(x.split(" ")) for x in docByUser]),
#      ', min= ',min([len(x.split(" ")) for x in docByUser]),
#      ', mean= ', np.mean([len(x.split(" ")) for x in docByUser]),
#        ', median= ', np.median([len(x.split(" ")) for x in docByUser])
#      )

## Create input sequences

In [27]:
review_sequences = []
for review in sents_train:
    tokenized_review = tokenizer.texts_to_sequences([review])[0] #map each word to word_index 
    #print(len(tokenized_review))
    for i in range(1, len(tokenized_review)): # make ngrams
        review_sequences.append( tokenized_review[:i+1] )
        
# get max length 
max_review_len = max([len(x) for x in review_sequences])
print(max_review_len)
        


190


In [0]:
#
review_sequences = np.array(pad_sequences(review_sequences,
                                         maxlen=max_review_len,
                                         padding='pre'))    
    
# generate set of last words in sequence
predictors, last_word = review_sequences[:,:-1] , review_sequences[:,-1]

In [29]:
len(lastword)
max(lastword)

500

In [0]:
last_word = tf.keras.utils.to_categorical(last_word, num_classes=max_vocab_words+1)

# LSTM Model

In [41]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(total_words, 100, input_length=max_review_len-1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences = True)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dense(max_vocab_words/2, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(max_vocab_words +1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 189, 100)          4433000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 189, 300)          301200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 189, 300)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_3 (Dense)              (None, 250)               25250     
_________________________________________________________________
dense_4 (Dense)              (None, 501)               125751    
Total params: 5,045,601
Trainable params: 5,045,601
Non-trainable params: 0
____________________________________________

In [0]:
history = model.fit(predictors, last_word, epochs=2, verbose = 1)

W0624 16:07:30.739965 140279235905408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
   7456/1523759 [..............................] - ETA: 26:53:33 - loss: 5.2893 - acc: 0.1977

In [39]:
last_word.shape

(1523759, 501)